## Getting data from the web

By [Allison Parrish](http://www.decontextualize.com/)

At this point, we have enough programming scaffolding in place to start talking about how to access Web APIs. A web API is some collection of data, made available on the web, provided in a format easy for computers to parse. But in order to write programs to access web APIs, I need to talk about a few other things first.

### URLs

A URL ("uniform resource locator") uniquely identifies a document on the web, and provides instructions for how to access it. It's the thing you type into your web browser's address bar. It's what you cut-and-paste when you want to e-mail an article to a friend. Most of what we do on the web---whether we're using a web browser or writing a program that accesses the web---boils down to manipulating URLs.

So it's important for us to understand the structure of URLs, so we can take them apart and put them back together (both in our heads and programmatically). URLs have a conventional structure that is specified in Internet standards documentation, and many of the web APIs we'll be accessing assume knowledge of this structure. So let's take the following URL:

    http://www.example.com/foo/bar?arg1=baz&arg2=quux
    
... and break it down into parts, so we have a common vocabulary.

| Part | Name |
|------|------|
| `http` | scheme |
| `www.example.com` | host |
| `/foo/bar` | path |
| `?arg1=baz&arg2=quux` | query string |

All of these parts are required, except for the query string, which is optional. Explanations:

* The *scheme* determines what *protocol* will be used to access this resource. For our purposes, this will almost always be `http` (HyperText Transfer Protocol) or `https` (HTTP, but over an encrypted connection).
* The *host* specifies which server on the Internet we're going to talk to in order to retrieve the document we want.
* The *path* names a resource on the server, often using slashes (`/`) to represent hierarchical relationships between resources. (Sometimes this corresponds to actual files on the server, but just as often it does not.)
* The *query string* is a means to tell the server *how* we want the document delivered. (More examples of this soon.)

Most of the work you'll do in learning how to use a web API is learning how to construct and manipulate URLs. A quick glance through [the documentation for, e.g., the Spotify API](https://developer.spotify.com/web-api/) reveals that the bulk of the documentation is just a big list of URLs, with information on how to adjust those URLs to get the information you want.

### HTML, JSON and web APIs

The most common format for documents on the web is HTML (HyperText Markup Language). Web browsers like HTML because they know how to render as human-readable documents---in fact, that's what web browsers are for: turning HTML from the web into visually splendid and exciting multimedia affairs.

HTML was specifically designed to be a tool for creating web pages, and it excels at that, but it's not so great for describing structured data. Another popular format---and the format we'll be learning how to work with this week---is JSON (JavaScript Object Notation). Like HTML, JSON is a format for exchanging structured data between two computer programs. Unlike HTML, JSON is primarily intended to communicate content, rather than layout.

Roughly speaking, whenever a web site exposes a URL for human readers, the document at that URL is in HTML. Whenever a web site exposes a URL for programmatic use, the document at that URL is in JSON. (There are other formats commonly used for computer-readable documents, like XML. But let's keep it simple for now.) As an example, Wordnik has a human-readable version of page for the definition of the word "hello," found at this URL:

> https://wordnik.com/words/hello

But Wordnik also has a version of this data designed to be easily readable by computers. This is the URL, and it returns a document in JSON format:

> http://api.wordnik.com:80/v4/word.json/hello/definitions

Every web site makes available a number of URLs that return human-readable documents; many web sites (like Twitter) also make available URLs that return documents intended to be read by computer programs. Often---as is the case with Facebook, or with sites like Metafilter that make their content available through RSS feeds---these are just two views into the same data.

You can think of a web API as the set of URLs, and rules for manipulating URLs, that a web site makes available and that are also intended to be read by computer programs. (API stands for "application programming interface"; a "web API" is an interface enables you to program applications that use the web site's data.)

### API Keys

You may have noticed when you clicked on the "computer-readable" link above that you received the following message (or one like it) in your web browser:

    {"message": "unauthorized", "type": "error"}

This message results from the fact that most web APIs (unlike most web pages) require some kind of *authentication*. "Authentication" here means some kind of information that associates the request with an individual. In many APIs, this takes the form of a "token" or "key" (also called a "client ID" and/or "secret")---most usually an extra parameter that you pass on the end of the URL (or in an HTTP header) that identifies the request as having come from a unique user. Some services (like Facebook) provide a subset of functionality to non-authenticated ("anonymous") requests; others require authentication for all requests.

So how do you get "keys" or "tokens"? There's usually some kind of sign-up form in or near the developer documentation for the service in question. Sign up for Wordnik [here](http://developer.wordnik.com/), or Foursquare [here](https://developer.foursquare.com/) (click on "My Apps" in the header after you've logged in). The form may ask you for a description of your application; it's usually safe to leave this blank, or to put in some placeholder text. Only rarely is this text reviewed by an actual human being; your key is usually issued automatically.

Different services have different requirements regarding how to include your keys in your request; you'll have to consult the documentation to know for sure.

In the following examples, I'll assume you've acquired an access token for the APIs in question and have assigned them to variables in the programs as appropriate.

### Fetching web documents with `requests`

All you need to make (most) web API requests is a web browser. But it would be tedious to do these requests in a web browser and then copy over the responses into Python for analysis. We may also want to make *many* requests to a web API (for example, to get hundreds of words from Wordnik), which is inconvenient to do "by hand" with a web browser. Ideally, there would be some way to make web requests *directly inside a Python program*.

Python does, in fact, have a built-in library ([urllib.request](https://docs.python.org/3/library/urllib.request.html)) just for this purpose! Actually, a number of other programmers have contributed their own libraries for making web requests in Python which improve on the capabilities of the built-in Python library. One of these is [requests](http://docs.python-requests.org/en/master/) which we'll be using below.

The purpose of the `requests` library is to make requests to web servers in order to retrieve web documents. You give it a URL, and it gives back a string that contains the content of the document located at that URL.

Here's an example of how to use `requests`. Our task here is to fetch the document at the URL given above, for the computer-readable version of Wordnik's definitions for the word "hello":

In [10]:
import requests

api_key = "a80a5131f7620c32a8919063dce09d01b6239543e3d0063bf"
url = "http://api.wordnik.com:80/v4/word.json/hello/definitions?api_key=" + api_key
response = requests.get(url)
data = response.json()
data

[{'attributionText': 'from The American Heritage® Dictionary of the English Language, 4th Edition',
  'citations': [],
  'exampleUses': [],
  'labels': [],
  'partOfSpeech': 'interjection',
  'relatedWords': [],
  'score': 0.0,
  'sequence': '0',
  'sourceDictionary': 'ahd-legacy',
  'text': 'Used to greet someone, answer the telephone, or express surprise.',
  'textProns': [],
  'word': 'hello'},
 {'attributionText': 'from The American Heritage® Dictionary of the English Language, 4th Edition',
  'citations': [],
  'exampleUses': [],
  'labels': [],
  'partOfSpeech': 'noun',
  'relatedWords': [],
  'score': 0.0,
  'sequence': '1',
  'sourceDictionary': 'ahd-legacy',
  'text': 'A calling or greeting of "hello.”',
  'textProns': [],
  'word': 'hello'},
 {'attributionText': 'from The American Heritage® Dictionary of the English Language, 4th Edition',
  'citations': [],
  'exampleUses': [],
  'labels': [],
  'partOfSpeech': 'verb-intransitive',
  'relatedWords': [],
  'score': 0.0,
  'se

Oh hey wow that's pretty rad! We'll break down *how* exactly to know what the URL for a particular resource is a bit later (and how to add the API to the request). But for now, just revel in the accomplishment of getting information from an API into your Python program.

Breaking down the code a little bit, the following lines are most important:

    response = requests.get(url)
    data = response.json()

This line calls the `get()` function in the `requests` package, with one parameter, the URL that you want to fetch (which we previously stored in a variable called `url`). When this function gets called, the `requests` library makes a network request to the specified URL and retrieves its contents, returning a special kind of value called a "response," which contains information about the response generated by the remote server, along with the content of that response. [Response objects have many different methods and attributes](http://docs.python-requests.org/en/master/api/#requests.Response), but the one we're most interested in is `.json()`, which takes data in the response in JSON format (if present) and converts it to the corresponding Python data structure.

The data returned from the Wordnik API is in JSON format. After being converted to a Python data structure, you can see it's a *list of dictionaries*. Using a `for` loop, we can print out the text of each definition:

In [11]:
for item in data:
    print(item['text'])

Used to greet someone, answer the telephone, or express surprise.
A calling or greeting of "hello.”
To call "hello.”


## Fun with the World Air Quality Index API

Many web sites and organizations offer web APIs. We're going to go over how one API in particular works, or at least a subset of a particular web API---the [World Air Quality Index API](http://aqicn.org/api/). The idea is that by introducing you to this one API, you'll learn the tools necessary to sign up for, query, and interpret APIs from other providers as well.

### Signing up for an API key

Before you can use the Air Quality Index API, you need to sign up for an API key. Do so by going to the API's [token request form](http://aqicn.org/data-platform/token/#/) and following the instructions. You'll need to provide your e-mail address. (When I signed up, I got my confirmation e-mail and API key very quickly!)

After confirming your e-mail address, you'll get your token. ("Token" in this context is just another word for "API key.") The token is just a string of letters and numbers. It'll look something like this:

    6617c28c371f0a138f7912a35365564afe538605
    
That's your "key" for that API. Whenever you make a request to that API, you'll need to include your key in the request. The exact methodology for including the key will be explained below. (Note: the key above is just something I made up; it's not a valid key; don't try using it in actual requests.)

### Looking at the documentation

The documentation for a web API usually describes what the API can do in terms of the URLs you can access, the patterns that those URLs follow, and the format of the responses. Here's a screenshot of the documentation page for the "search" function of the World Air Quality Index API, for example:

![aqicn docs](http://static.decontextualize.com/snaps/aqicn-doc-snap.png)

This is a little confusing when you first look at it, but it's telling you the following:

* The box under "GET" shows you what the URL path should look like to make this request.
* Inside the path, the words preceded by colons (like `:keyword`) are intended to be *placeholders*: you should replace those with the values that correspond to what you want the request to do.
* The `Parameter` table shows what the placeholder values mean. (Don't worry about the `callback` parameter—that's only useful if you're a Javascript programmer.)
* The `Success 200` field shows what the *response* looks like. It tells you the names and types of the values in the JSON object returned in the response.

Let's make a quick test request.

In [30]:
token = "" # replace this with your own key
url = "https://api.waqi.info/search/"
resp = requests.get(url, params={"token": token, "keyword": "shenzhen"})
data = resp.json()
data

{'data': [{'aqi': '70',
   'station': {'geo': [22.543099, 114.057868],
    'name': 'Shenzhen',
    'url': 'shenzhen'},
   'time': {'stime': '2017-07-19 22:00:00',
    'tz': '+0800',
    'vtime': 1500472800},
   'uid': 1539},
  {'aqi': '42',
   'station': {'geo': [22.542454, 113.987495],
    'name': 'OCT, Shenzhen',
    'url': 'shenzhen/huaqiaocheng'},
   'time': {'stime': '2017-07-19 22:00:00',
    'tz': '+0800',
    'vtime': 1500472800},
   'uid': 927},
  {'aqi': '',
   'station': {'geo': [0, 0], 'name': 'Xiapi, Shenzhen', 'url': ''},
   'time': {'stime': '', 'tz': '', 'vtime': 0},
   'uid': 915},
  {'aqi': '',
   'station': {'geo': [0, 0], 'name': 'Fuyong, Shenzhen', 'url': ''},
   'time': {'stime': '', 'tz': '', 'vtime': 0},
   'uid': 932},
  {'aqi': '30',
   'station': {'geo': [22.5500005, 114.0960791],
    'name': 'Liyuan, Shenzhen',
    'url': 'shenzhen/liyuan'},
   'time': {'stime': '2017-07-19 22:00:00',
    'tz': '+0800',
    'vtime': 1500472800},
   'uid': 925},
  {'aqi': '50

This request is finding all air quality stations by name using the search string `shenzhen`. As described in the documentation, the results take the form of a dictionary with two keys: `data` (which points to a list of dictionaries) and `status`, whose value is a string (`'ok'`). Each one of the items in the list that is the value for key `data` is itself a dictionary. Important items in this dictionary are the air quality index itself (`aqi`), the ID of the station (`uid`) and the `station` key, which points to *another* dictionary with the name of the station (`name`) and a `url` field, that can be used with a different API endpoint to get more detail about a particular station. Using just this data, we could write a loop to make a simple report on our findings:

In [37]:
print("Station name".ljust(30, " "), "AQI")
print("-" * 30, "---")
for item in data['data']:
    print(item['station']['name'].ljust(30, " "), item['aqi'])

Station name                   AQI
------------------------------ ---
Shenzhen                       70
OCT, Shenzhen                  42
Xiapi, Shenzhen                
Fuyong, Shenzhen               
Liyuan, Shenzhen               30
Honghu, Shenzhen               50
Nanyou, Shenzhen               53
Yantian, Shenzhen              42
Longhua, Shenzhen              
Henggang, Shenzhen             
Xi xiang, Shenzhen             42
Guangming, Shenzhen            
Nan'aozhen, Shenzhen           70
Kwai Chung, Shenzhen           38


> Exercise: To better understand the code above, research the `.ljust()` method and the `*` operator as it applies to strings. (This has nothing to do with how we got the data, just how we displayed it.)

> Exercise 2: Perform the search above on a different city.

> Exercise 3: Write code that finds the latitude of the northernmost station in the search results. (Hint: use the `min()` function and a list comprehension.

### Building query strings dynamically

So, one quick thing: what's with that `params` thing in the call to `requests.get()`? Well, it's there to make our lives easier. If you put `params=` into the arguments of the call to `requests.get()`, the library will take the dictionary that you passed and use it to form a query string. It does this so you don't have to figure out how to write the query string yourself.

Let's review what a query string looks like. Here's the example query string from the section above about URL structure:

    ?arg1=baz&arg2=quux
    
At first glance, it looks like garbage. For another perspective, let's look at the query string from the example request to the Air Quality API:

    ?token=6617c28c371f0a138f7912a35365564afe538605&keyword=salt+lake+city
    
A little bit of the structure becomes more apparent here! It looks like we have a series of key/value, separated by ampersands (`&`): `token=6617c28c371f0a138f7912a35365564afe538605` and `keyword=salt+lake+city`. The pairs themselves are separated by equal signs (`=`).

> INTEPRETIVE QUESTION: What kind of data structure does this resemble? It's a data structure that we talked about earlier today. Think hard. Yes, it's a dictionary!

What we'd like to have, then, is some kind of way to write an expression that turns a Python dictionary into a string formatted correctly to include in a query string. It turns out that the process of doing this is [kind of tricky](http://en.wikipedia.org/wiki/Percent-encoding), so the `requests` library does the hard work for us when we pass the parameters to the `requests.get()` function with the named `params` parameter. Very handy!

### Working with responses

Now we've gotten a response from the API, and we've parsed it into a Python data structure that we know how to use (a dictionary). But now what do we do with it?

Let's look at the `/feed` resource for the Air Quality API. According to the documentation, we can make a request to get air quality information about a particular city by accessing a path with the following format:

    /feed/:city/?token=:token
    
The formatting here with the colon (`:`) isn't Python syntax—it's just a shorthand that programmers sometimes use to mean "fill this in with the corresponding value." So `:city` in the API documentation for this API means "replace this with the city you want" and `:token` means "replace this with your token." According to the API, you can replace `:city` with either the name of the city, or the ID of the city, both of which are returned in the search results we retrieved above. The ID is a bit less ambiguous, so I'm going to use that in the subsequent examples. (The ID is in the search results as the value for key `uid`. The documentation also states that if you use the ID, you need to precede it with a `@` symbol, which is reflected in the code below.)

First off, let's look at the actual structure of the data that we have and try to characterize its structure, from both a syntactic (what are the parts and what's it made of) and semantic (what does it mean?) perspective.

One of Juputer Notebook's nice features is that if you make a code cell with a variable or expression on its own in a single line, it will do its best to format the value of that expression in a nice, readable way. Let's do this for the `data` variable that we created in a cell above.

In [96]:
url = "https://api.waqi.info/feed/@1539/"
resp = requests.get(url, params={"token": token})
data = resp.json()
data

{'data': {'aqi': 59,
  'attributions': [{'name': 'Shenzhen Environment Network (深圳人居环境网)',
    'url': 'http://www.szhec.gov.cn/'},
   {'name': 'Guangdong Environmental Protection public network (广东环境保护公众网)',
    'url': 'http://www.gdep.gov.cn/'}],
  'city': {'geo': [22.543099, 114.057868],
   'name': 'Shenzhen',
   'url': 'http://aqicn.org/city/shenzhen/'},
  'dominentpol': 'pm25',
  'iaqi': {'co': {'v': 6.5},
   'd': {'v': 27},
   'h': {'v': 79},
   'no2': {'v': 7.4},
   'o3': {'v': 25.6},
   'p': {'v': 1009},
   'pm10': {'v': 22},
   'pm25': {'v': 59},
   'so2': {'v': 4.6},
   't': {'v': 31},
   'w': {'v': 3.1},
   'wd': {'v': 250}},
  'idx': 1539,
  'time': {'s': '2017-07-20 13:00:00', 'tz': '+08:00', 'v': 1500555600}},
 'status': 'ok'}

Okay, this is still a little bit unreadable. I'm going to do my best to parse it out.

> NOTE: nearly every API has its own idiosyncratic way of structuring its responses. Part of the point of API documentation is to let programmers know how the response is structured and what the response means. There's no substitute for studying the API documentation, but with a bit of practice, you can usually heuristically pinpoint which parts of a response are interesting based merely on what's visible in the response. 

The Python data structure here is a dictionary. We know that from looking at it, but also because of what happens when we evaluate this expression:

In [44]:
type(data)

dict

The dictionary appears to have two keys. Here are my guesses about what their values must mean, based on my experience and guesses:

* `data` is pointing to a key whose value is... well, the actual data that we want. It's not uncommon for responses from APIs to have a top-level nested structure like this, where the relevant "content" is hidden one level deep in the hierarchy.
* `status` appears to be some kind of... status message. I assume everything is okay because it says `ok`.

The key `data` points to a dictionary, and within that dictionary we find the data we actually want. Within *that* dictionary, there are several keys of interest:

* `aqi` appears to be the general "air quality index";
* `dominentpol` appears to be the dominant pollutant;
* `iaqi` is another dictionary, with keys for (I assume) different pollutants. Each one of those keys—somewhat inexplicably—has *another* dictionary for its values, whose only key (`v`) points to the actual value for that pollutant.

Based on this analysis, I'm going to hone in on the value for the `data` key as something to play with. Let's confirm our suspicion that the value for this key is a dictionary:

In [45]:
type(data['data'])

dict

Okay, so what's the value for the `aqi` key of that dictionary?

In [49]:
type(data['data']['iaqi'])

dict

...and what's the value for the key `pm25` item in that dictionary?

In [51]:
type(data['data']['iaqi']['pm25'])

dict

A dictionary! So we have a dictionary... with another dictionary inside of it... with another dictionary inside of it. To get the value of the key for the expression above:

In [57]:
data['data']['iaqi']['pm25']['v']

70

With the expression above, we can swap out `pm25` with any of the other keys listed in the `iaqi` dictionary. So, for example, to get the value for ozone (`o3`):

In [58]:
data['data']['iaqi']['o3']['v']

11

Armed with this knowledge, we can write a `for` loop that prints out the value for all listed pollutants. At a glance we can see that each of the dictionaries that are the values for the keys in the `iaqi` dictionary have the same structure, so we can loop over the keys of the `iaqi` dictionary, and get the value with the same expression:

In [60]:
for key in data['data']['iaqi'].keys():
    print(key, data['data']['iaqi'][key]['v'])

co 8.2
d 26
h 89
no2 18.8
o3 11
p 1009
pm10 32
pm25 70
so2 4.1
t 28
w 0.9
wd 270


## Full example: comparing pollutants of different cities

In this example, I want to *compare* the pollutants of multiple cities. To do this, we need to do multiple requests: one request to the "feed" for each of the cities in our list. In order to do this, however, we need to do some preliminary work: find the appropriate string to use in the "feed" URL for the city in question. The easiest way to find this is with the API's `search` resource, which we already experimented above. I wrote a little list comprehension below that shows the name and ID of each station included in the search results. The second item of the tuple is what you'd add to the list of cities below.

In [105]:
url = "https://api.waqi.info/search/"
resp = requests.get(url, params={"token": token, "keyword": "shanghai"})
data = resp.json()
[(item['station']['name'], item['uid']) for item in data['data']]

[('Shanghai Normal University, Shanghai', 487),
 ('Shanghai Normal College Primary Division, Shanghai', 483),
 ('Shanghai', 1437),
 ('Shanghai US Consulate, Shanghai', 3304),
 ('Putuo, Shanghai', 481),
 ('Jingan, Shanghai', 486),
 ('Zhangjiang, Shanghai', 489),
 ('City average, Shanghai', 491),
 ('Hongkou Liangcheng, Shanghai', 485),
 ('杨浦四漂, Shanghai; Yangpu Sipiao, Shanghai', 482),
 ('浦东川沙, Shanghai; Chuansha, Shanghai', 488),
 ('青浦淀山湖, Shanghai; Dianshan Lake, Shanghai', 484),
 ('浦东监测站, Shanghai; Lingshan Road, Shanghai', 490)]

I want to compare pollution for the following four cities, whose strings I found using the code above:

In [106]:
cities = [
    "1539", # shenzhen
    "1451", # beijing
    "1437", # shanghai
    "3309", # new york
    "243", # los angeles
    "3189", # london
]

I'm going to issue a web request for each of these cities' AQIs and create a list of lists of the results. The data structure I want to end up with looks like this:

    [["Shenzhen", 123],
     ["Beijing", 123],
     ["Shanghai", 123],
     ["New York", 123],
     ["Los Angeles", 123],
     ["London", 123]]
     
... where `123` would be the current AQI for that city. Let's break this down by writing the code we'd need for just *one* row in this table.

In [102]:
city = "1539" # set which city we want to query
url = "https://api.waqi.info/feed/@" + city + "/" # build URL with path based on city
resp = requests.get(url, params={"token": token}) # perform web request
data = resp.json() # get the data
city_name = data['data']['city']['name'] # city's name is stored in the 'city' dictionary
aqi = data['data']['aqi'] # city's AQI stored in 'aqi' key
row = [city_name, aqi] # create the row
row

['Shenzhen', 59]

To do this for *all of the cities in our list*, we'll copy that code and indent it over in a `for` loop. At the end of the loop, instead of just evaluating the row for display, we'll *append* it to a list called `table`:

In [107]:
table = []
for city in cities:
    url = "https://api.waqi.info/feed/@" + city + "/"
    resp = requests.get(url, params={"token": token})
    data = resp.json()
    city_name = data['data']['city']['name']
    aqi = data['data']['aqi']
    row = [city_name, aqi]
    table.append(row)

Nice! Now we have a list of lists of these results:

In [109]:
table

[['Shenzhen', 59],
 ['Beijing', 352],
 ['Shanghai', 93],
 ['New York', 70],
 ['Los Angeles-North Main Street', 55],
 ['London Bloomsbury', 47]]

If you get an error with any of these calls, a helpful thing to do is use the `print()` function inside the loop to display the response you received from the API. Often the response itself will tell you what went wrong (e.g., you asked for an ID that didn't exist).

### Exporting to CSV

Now that we have the data from the API, it might be desirable to be able to export our data so we can use it in other tools. We'll use the `csv` module's `writer` object to accomplish this.

In [111]:
import csv
with open("aqi_export.csv", "w") as filehandle:
    writer = csv.writer(filehandle)
    writer.writerow(["city", "aqi"])
    for row in table:
        writer.writerow(row)

The `with` clause may look new to you! We won't go over the technical details here ([though you can read a good introduction here](http://effbot.org/zone/python-with-statement.htm)). Essentially, the line starting with the `with` keyword creates a new file object with the filename `aqi_export.csv` and assigns it to a variable called `filehandle`.

In the indented section beneath, we create a special kind of Python value called a "CSV writer" (from the csv module), passing it the file object as a parameter. The writer object has one method of importance to us, .writerow(), which takes a list of values. The writer object takes the values passed to .writerow(), formats them by converting them to strings and inserting commas between them, and writes them to the file opened in the initial line of the with clause. The first call to .writerow() writes the header row, which our spreadsheet software will interpret as column names. Inside a `for` loop, we visit each row from the `table` list made in the previous step and write it to the file.

The resulting file (`aqi_export.csv`) will be located in the same directory as your Jupyter Notebook. It'll look something like this:

    city,aqi
    Shenzhen,59
    Beijing,352
    Shanghai,93
    New York,70
    Los Angeles-North Main Street,55
    London Bloomsbury,47

Now you can open it in a spreadsheet program of your choice! I opened it with Numbers and made this sweet graph:

![AQI graph with data on Shenzhen, Beijing, Shanghai, New York, Los Angeles, London](http://static.decontextualize.com/snaps/aqi_graph.png)

> Exercise: Add a few more cities to the `cities` list to include in the graph. You'll need to use the `search` resource in the AQI API to find the appropriate ID!

> Exercise 2: Add a few more *columns* to the data---say, the values for different types of pollutants. This one's tricky but worthwhile! For example, [here's a graph I made with a couple other pollutant types](http://static.decontextualize.com/snaps/aqi_graph2.png). I'm not saying this is a *good* graph. But it was a good exercise.